### 数据清洗：对重复值、缺失值和异常值的处理

In [1]:
%matplotlib inline

#### 缺失值处理

In [14]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import Imputer
# Imputer类在sklearn V0.20及以后的版本被impute.SimpleImputer类取代
from sklearn.impute import SimpleImputer

In [4]:
# 生成缺失值
df = pd.DataFrame(np.random.randn(6, 4), columns=['col1', 'col2', 'col3', 'col4'])
df.iloc[1:2, 1] = np.nan
df.iloc[4, 3] = np.nan
df

,col1,col2,col3,col4
0,-0.500350,0.651238,0.731450,-0.465849
1,-0.109416,NaN,0.664389,-1.083839
2,-1.093254,1.976900,-0.339053,-0.446078
3,-1.448660,1.274744,0.539189,-0.423813
4,-0.601059,0.096444,-1.419141,NaN
5,0.057581,1.747570,1.422875,0.263999


In [25]:
# 查看缺失值
nan_all = df.isnull()

# 查看哪些列有缺失值
nan_col1 = df.isnull().any()  # 含有NA的列
nan_col2 = df.isnull().all()  # 全部为NA的列

# 丢弃缺失值
df_dropna = df.dropna()  # 丢掉包含缺失值的行记录

# 使用sklearn的数据预处理方法对缺失值处理
nan_model = SimpleImputer(missing_values=np.nan, strategy='mean')  # 建立替换规则，将NAN用特征均值替换
nan_result = nan_model.fit_transform(df)  # 应用模型规则

#  使用pandas处理缺失值，方法是DataFrame.fillna()，该方法主要的两个参数value和method，value通过固定值替换缺失值，method用pandas提供的方法替换
nan_result_pd1 = df.fillna(0)  # 用0替换缺失值
nan_result_pd2 = df.fillna(method='pad')  # 用前面的值替换nan,或用ffill
nan_result_pd3 = df.fillna(method='backfill')  #用后面的值替换，或用bfill
nan_result_pd4 = df.fillna(df.mean()['col2':'col4'])  # 用该列的均值替换

,col1,col2,col3,col4
0,-0.500350,0.651238,0.731450,-0.465849
1,-0.109416,1.149379,0.664389,-1.083839
2,-1.093254,1.976900,-0.339053,-0.446078
3,-1.448660,1.274744,0.539189,-0.423813
4,-0.601059,0.096444,-1.419141,-0.431116
5,0.057581,1.747570,1.422875,0.263999


#### 异常值处理
异常值的确定常采用Z-Score标准化作为阈值--判断标准，标准化的得分超出阈值则为异常

In [35]:
df_outlier = pd.DataFrame({'col1':[1, 120, 3, 5, 2, 12, 13],
                           'col2':[12, 17, 31, 53, 22, 32, 43]})

In [52]:
# 将其Z-Score标准化
df_zscore = df_outlier.copy()
cols = df_zscore.columns
df_z = pd.DataFrame()
for col in cols:
    df_col = df_zscore[col]
    z_score = (df_col - df_col.mean()) / df_col.std()  # 求该列的z-score得分
    df_z[col] = z_score.abs() > 2.2

df_z_score = pd.concat([df_zscore, df_z], axis=1)
df_z_score

,col1,col2,col1,col2
0,1,12,False,False
1,120,17,True,False
2,3,31,False,False
3,5,53,False,False
4,2,22,False,False
5,12,32,False,False
6,13,43,False,False


#### 重复值的处理

In [65]:
# 生成重复数据
data1, data2, data3, data4 = ['a', 3], ['b', 2], ['a', 3], ['c', 2]
df_repeat = pd.DataFrame([data1, data2, data3, data4], columns=['col1', 'col2'])

is_duplicated = df_repeat.duplicated()  # 判断重复值
df_norepeat = df_repeat.drop_duplicates()  # 删除数据记录中所有列值相同的记录
df1_norepeat = df_repeat.drop_duplicates(['col1'])  # 删除数据记录中col1列值相同的记录


,col1,col2
0,a,3
1,b,2
3,c,2


### 数据转换
对非数值型数据(分类数据和顺序数据)转化为数值型数据

In [82]:
from sklearn.preprocessing import OneHotEncoder

# 生成数据
df_classify = pd.DataFrame({'id': [3566841, 6541227, 3512441],
                   'sex': ['male', 'Female', 'Female'],
                   'level': ['high', 'low', 'middle'],
                   'score': [1, 2, 3]})

# 将ID和数据列分开
id_data = df_classify[['id']]
raw_convert_data = df_classify.iloc[:, 1:]

model_enc = OneHotEncoder()  # 建立标志转换模型对象，即哑编码对象
df_cl = model_enc.fit_transform(raw_convert_data).toarray()  # 标志转换,并将其以矩阵形式输出，对score列也做转换
df_all = pd.concat([id_data, pd.DataFrame(df_cl)], axis=1)  # 将转换后的矩阵与id合并

df_cl1 = pd.get_dummies(raw_convert_data)  # 使用get_dummies()方法仅对object和category类型的数据列做标志转换,不对score列做转换
df_all2 = pd.concat([id_data, pd.DataFrame(df_cl1)], axis=1)  # 将转换后的矩阵与id合并
df_all2

,id,score,sex_Female,sex_male,level_high,level_low,level_middle
0,3566841,1,0,1,1,0,0
1,6541227,2,1,0,0,1,0
2,3512441,3,1,0,0,0,1
